# This is the code that I used to generate my Capstone Project
##  "Dinner and a Date"
### Jerry Ackaret

In [69]:
import pandas as pd
import numpy as np
import requests 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [70]:
#Set some variables
radius = 750  #radius in meters to find food venues around the fun venues
Number_FoodperFun = 10 # Maximum number of food venues to return for each fun venue

In [71]:
#Determine the lat/long for Portland, OR for use in the mapping, using geocode
address = 'Portland, OR'

geolocator = Nominatim(user_agent="port_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Portland are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Portland are 45.5202471, -122.6741949.


In [72]:
CLIENT_ID = 'PC5PSKF4U55ZE4AXQDJH53V4PYE5PPHIUEEDGWECNJ3J2YQH' # your Foursquare ID
CLIENT_SECRET = 'U5FX4H5XEHPSEN02VM1CDUE4GC133EZBUF0VCJIOWLUDRW2G' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: PC5PSKF4U55ZE4AXQDJH53V4PYE5PPHIUEEDGWECNJ3J2YQH
CLIENT_SECRET:U5FX4H5XEHPSEN02VM1CDUE4GC133EZBUF0VCJIOWLUDRW2G


In [73]:
#Get the Fun Venues out of Foursquare, limiting the results to ones that I want for Dinner and a Date baskets
LIMIT = 100

near = 'Portland, OR'
url = 'https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d181941735,4bf58dd8d48988d1e2931735,5032792091d4c4b30a586d5c,4deefb944765f83613cdba6e,4bf58dd8d48988d184941735,4bf58dd8d48988d17b941735&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    near, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d181941735,4bf58dd8d48988d1e2931735,5032792091d4c4b30a586d5c,4deefb944765f83613cdba6e,4bf58dd8d48988d184941735,4bf58dd8d48988d17b941735&client_id=PC5PSKF4U55ZE4AXQDJH53V4PYE5PPHIUEEDGWECNJ3J2YQH&client_secret=U5FX4H5XEHPSEN02VM1CDUE4GC133EZBUF0VCJIOWLUDRW2G&v=20190605&near=Portland, OR&limit=100'

In [74]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da39a1602a172001b839370'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'portland or',
   'center': {'lat': 45.52345, 'lng': -122.67621},
   'displayString': 'Portland, OR, United States',
   'cc': 'US',
   'geometry': {'bounds': {'ne': {'lat': 45.653272, 'lng': -122.472021},
     'sw': {'lat': 45.432393, 'lng': -122.83675}}},
   'slug': 'portland-oregon',
   'longId': '72057594043674481'},
  'headerLocation': 'Portland',
  'headerFullLocation': 'Portland',
  'headerLocationGranularity': 'city',
  'query': 'museum',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 45.5711550870764,
    'lng': -122.45622999557082},
   'sw': {'lat': 45.503355322651565, 'lng': -122.8797003740795}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is p

In [75]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [76]:
#Build the DataFrame of the Fun Venues
venues = results['response']['groups'][0]['items']
    
fun_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.address','venue.id']
fun_venues =fun_venues.loc[:, filtered_columns]

# filter the category for each row
fun_venues['venue.categories'] = fun_venues.apply(get_category_type, axis=1)

# clean columns
fun_venues.columns = [col.split(".")[-1] for col in fun_venues.columns]

fun_venues.head()

,name,categories,lat,lng,address,id
0,Portland Art Museum,Art Museum,45.516230,-122.683597,1219 SW Park Ave,43ee1bd9f964a5205f2f1fe3
1,Pittock Mansion,Museum,45.525262,-122.716684,3229 NW Pittock Dr,41b8e700f964a520801e1fe3
2,Oregon Museum of Science & Industry (OMSI),Science Museum,45.508601,-122.665915,1945 SE Water Ave,415c9e00f964a520551d1fe3
3,Portland Children's Museum,Science Museum,45.508530,-122.717786,4015 SW Canyon Rd,4efdf6cea69d45461bd1cf1a
4,Oregon Historical Society,History Museum,45.515914,-122.682456,1200 SW Park Ave,4b5ce375f964a5207a4929e3


In [77]:
# create map of Portland using latitude and longitude values
map_portland = folium.Map(location=[latitude, longitude], zoom_start=11)

# add the Fun Venues markers to map
for lat, lng, name in zip(fun_venues['lat'], fun_venues['lng'], fun_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_portland)  
    
map_portland


In [78]:
#Get rid of the items outside of City
fun_venues = fun_venues[fun_venues['name'] != 'Washington County Museum']
fun_venues = fun_venues[fun_venues['name'] != 'Madrugada Pottery']
print('{} venues were returned by Foursquare.'.format(fun_venues.shape[0]))

28 venues were returned by Foursquare.


In [79]:
#Function to add the Food Venues that surround the Fun Venues
def getNearbyVenues(names, latitudes, longitudes, f_address, fun_id):
    
    venues_list=[]
    for name, lat, lng, faddress, funid in zip(names, latitudes, longitudes, f_address, fun_id):
        print(name, end=", ")
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=food&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            faddress,
            lat, 
            lng,
            funid,
            a['venue']['name'], 
            a['venue']['id'],
            a['venue']['location']['lat'], 
            a['venue']['location']['lng'],  
            a['venue']['location']['distance'],
            a['venue']['categories'][0]['name'],
            a['venue']['location']['formattedAddress']) for a in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
#Change the column headings to be more descriptive
    nearby_venues.columns = ['Fun_Venue', 
                  'Fun_Address',           
                  'Fun_Latitude', 
                  'Fun_Longitude', 
                  'Fun_ID',           
                  'Venue', 
                  'Venue_ID',
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Distance',           
                  'Venue_Category', 
                  'Venue_Address']
    
    return(nearby_venues)

In [80]:
#Calling the function to get all of the Food Venues around the Fun Venues
Fun_food_venues = getNearbyVenues(names=fun_venues['name'],
                                   latitudes=fun_venues['lat'],
                                   longitudes=fun_venues['lng'],
                                   f_address=fun_venues['address'],
                                   fun_id=fun_venues['id']
                                  )

Portland Art Museum, Pittock Mansion, Oregon Museum of Science & Industry (OMSI), Portland Children's Museum, Oregon Historical Society, Oregon History Museum, Harry C. Kendall Planetarium, USS Blueback, Oregon Rail Heritage Center, Newspace Center for Photography, World Forestry Center, OMSI After Dark, Turbine Hall (OMSI), Oregon Maritime Museum, Robot Exhibit At Omsi, Research Library, Oregon Nikkei Legacy Center, Hat Museum, Science Store @ OMSI, Portland Chinatown Museum, Doll Asylum, Architectural Heritage Center, The Zymoglyphic Museum, Portland Mayor’s Mansion, History Hub, PIXAR Exhibition, Historic Belmont Firehouse, Oregon Jewish Museum & Center for Holocaust Education (OJMCHE), 

In [81]:
Fun_food_venues.shape

(1561, 12)

In [82]:
#Lets get rid of the real cheap eateries
Fun_good_venues=Fun_food_venues[~Fun_food_venues['Venue_Category'].isin(
    ['Food Truck','Donut Shop','Bakery','Bagel Shop','Coffee Shop','Dessert Shop','Fast Food Restaurant',
     'Food Court','Food Stand','Juice Bar','Snack Place'])]

In [83]:
# get dataframe sorted by Venue Distance for each Fun Venue with limit returned set by Number_FoodperFun at beginning of program

Top = Fun_good_venues.groupby(["Fun_Venue"]).apply(lambda x: x.sort_values(["Venue_Distance"], ascending = True)).reset_index(drop=True)
# select closest eating establishments
Top_df=Top.groupby('Fun_Venue').head(Number_FoodperFun)

#Set up a list of just the resultant food venues IDs
Just_food_venues_wdup = Top_df[['Venue_ID']]
Just_food_venues=Just_food_venues_wdup.drop_duplicates(keep='last')

In [86]:
def getVenueDetails(IDs): #Function to retrieve the Phone and Rating (if any) for a list of Venue_IDs
    
    venue_details_list=pd.DataFrame(columns=['id','contact.phone'])

    for Venue_ID in (IDs):         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            Venue_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)

        resultsofdetails = requests.get(url).json()['response']['venue']
        resultsofdetails2=json_normalize(resultsofdetails) #Flatten JSON 
        
        # filter columns
        filtered_columns = ['id','contact.phone','rating']
        resultsofdetails3=pd.DataFrame(resultsofdetails2.loc[:,filtered_columns])        
        venue_details_list = pd.concat([venue_details_list, resultsofdetails3],sort=True)
    
    My_venues = venue_details_list
    My_venues.columns = ['Venue_Phone',
                         'Venue_ID',
                         'Venue_Rating'
                       ]
    return(My_venues)

In [87]:
# Retrieve the Food Venue Phones and Ratings
Food_Phone_Rating = getVenueDetails(IDs=Just_food_venues['Venue_ID']
                        )

In [88]:
# Retrieve the Fun Venues Phones and Ratings
Fun_Phone_Rating = getVenueDetails(IDs=fun_venues['id']
                        )
Fun_Phone_Rating.columns=['Fun_Phone', 'Fun_ID', 'Fun_Rating']

In [89]:
#Merge Both Phone and Ratings with the list of Fun Venues with Food Venues
Top_df_merge1 = pd.merge(Top_df, Fun_Phone_Rating, left_on='Fun_ID', right_on='Fun_ID')

Top_df_merge2 = pd.merge(Top_df_merge1, Food_Phone_Rating, left_on='Venue_ID', right_on='Venue_ID')

col_index=[0,1,12,13,2,3,4,5,10,11,14,15,9,7,8,6]
Top_df_merge_reorder=Top_df_merge2[[Top_df_merge2.columns[i] for i in col_index]]
Top_df_merge_resort= Top_df_merge_reorder.sort_values(['Fun_Venue', 'Venue_Rating'])

In [90]:
#Check the output for functionality
Top_df_merge_resort.tail()

,Fun_Venue,Fun_Address,Fun_Phone,Fun_Rating,Fun_Latitude,Fun_Longitude,Fun_ID,Venue,Venue_Category,Venue_Address,Venue_Phone,Venue_Rating,Venue_Distance,Venue_Latitude,Venue_Longitude,Venue_ID
75,USS Blueback,1945 SE Water Ave,NaN,7.9,45.508422,-122.666018,4b48e783f964a520a85c26e3,Portland Spirit Cruises and Events,Restaurant,"[110 SE Caruthers St, Portland, OR 97214, Unit...",5032243900,NaN,328,45.505687,-122.664433,5b07770b4420d8002c39dade
111,USS Blueback,1945 SE Water Ave,NaN,7.9,45.508422,-122.666018,4b48e783f964a520a85c26e3,Newport Floating Restaurant,Seafood Restaurant,"[Portland, OR, United States]",NaN,NaN,465,45.509470,-122.671792,51a3b9a0498e1de15d2c25e7
242,World Forestry Center,4033 SW Canyon Rd,5032281367,7.5,45.510465,-122.718061,49cc4846f964a5205e591fe3,AfriCafe,Café,"[at Oregon Zoo, Portland, OR 97221, United Sta...",NaN,4.9,441,45.508541,-122.713114,4bd9e6762a3a0f47c5cba8b6
238,World Forestry Center,4033 SW Canyon Rd,5032281367,7.5,45.510465,-122.718061,49cc4846f964a5205e591fe3,Cascade Grill,Restaurant,"[SW Zoo Rd, Portland, OR, United States]",NaN,5.2,186,45.510025,-122.715750,4bae7004f964a52060b23be3
240,World Forestry Center,4033 SW Canyon Rd,5032281367,7.5,45.510465,-122.718061,49cc4846f964a5205e591fe3,Bear Walk Cafe,Café,"[SW Zoo Rd, Portland, OR, United States]",NaN,NaN,374,45.509234,-122.713589,4e04cca0e4cd4839a4d4d1ae


In [91]:
#Get a version of the output that will be used for the spreadsheet, removing unneeded columns
Top_cleaned = Top_df_merge_resort.drop(['Fun_Latitude', 'Fun_Longitude', 'Fun_ID', 'Venue_Latitude', 'Venue_Longitude', 'Venue_ID'],axis=1)

In [93]:
# Make the Excel Spreadsheet
Top_cleaned.to_excel("Solicitation.xlsx")

In [94]:
# create map of Portland using latitude and longitude values
map_portland = folium.Map(location=[latitude, longitude], zoom_start=12)

# add the Fun Venues markers to map
for lat, lng, name in zip(fun_venues['lat'], fun_venues['lng'], fun_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_portland)  
    
for lat2, lng2, name in zip(Top_df_merge_resort['Venue_Latitude'], Top_df_merge_resort['Venue_Longitude'], Top_df_merge_resort['Venue']):  
    label2 = '{}'.format(name)
    label2 = folium.Popup(label2, parse_html=True)    
    folium.features.CircleMarker(
        [lat2, lng2],
        radius=3,
        popup=label2,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False).add_to(map_portland)
map_portland    

In [95]:
#How many Food Venues are there for each of the Fun Venues
Fun_good_venues.groupby('Fun_Venue').count()

,Fun_Address,Fun_Latitude,Fun_Longitude,Fun_ID,Venue,Venue_ID,Venue_Latitude,Venue_Longitude,Venue_Distance,Venue_Category,Venue_Address
Fun_Venue,,,,,,,,,,,
Architectural Heritage Center,65,65,65,65,65,65,65,65,65,65,65
Doll Asylum,15,15,15,15,15,15,15,15,15,15,15
Harry C. Kendall Planetarium,30,30,30,30,30,30,30,30,30,30,30
Hat Museum,43,43,43,43,43,43,43,43,43,43,43
Historic Belmont Firehouse,51,51,51,51,51,51,51,51,51,51,51
History Hub,79,79,79,79,79,79,79,79,79,79,79
Newspace Center for Photography,58,58,58,58,58,58,58,58,58,58,58
OMSI After Dark,25,25,25,25,25,25,25,25,25,25,25
Oregon Historical Society,79,79,79,79,79,79,79,79,79,79,79


Some of the Fun Venues are lacking in a lot of Food choices, even with a 750 meter circle.  Which is why the radius is so large.  Note:  Pitcock Mansion is not even listed ... no Food venues in the area at all.

In [96]:
# one hot encoding of the food venues that have the "cheap" ones removed
Fun_Food_onehot = pd.get_dummies(Fun_good_venues[['Venue_Category']], prefix="", prefix_sep="")

# add Fun_Venue column back to dataframe
Fun_Food_onehot['Fun_Venue'] = Fun_good_venues['Fun_Venue'] 

# move Fun_Venue column to the first column
fixed_columns = [Fun_Food_onehot.columns[-1]] + list(Fun_Food_onehot.columns[:-1])
Fun_Food_onehot = Fun_Food_onehot[fixed_columns]

Fun_Food_onehot.head()


,Fun_Venue,American Restaurant,Asian Restaurant,BBQ Joint,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Café,Cajun / Creole Restaurant,Chinese Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Diner,Eastern European Restaurant,Ethiopian Restaurant,Fish & Chips Shop,Fondue Restaurant,Food,French Restaurant,Gastropub,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Mongolian Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Soup Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Portland Art Museum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Portland Art Museum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Portland Art Museum,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Portland Art Museum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Portland Art Museum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [97]:
Fun_Food_onehot.shape

(1253, 63)

In [98]:
Fun_Food_grouped = Fun_Food_onehot.groupby('Fun_Venue').mean().reset_index()
Fun_Food_grouped


,Fun_Venue,American Restaurant,Asian Restaurant,BBQ Joint,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Café,Cajun / Creole Restaurant,Chinese Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Diner,Eastern European Restaurant,Ethiopian Restaurant,Fish & Chips Shop,Fondue Restaurant,Food,French Restaurant,Gastropub,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Mongolian Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Peruvian Restaurant,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Soup Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Architectural Heritage Center,0.030769,0.000000,0.000000,0.015385,0.000000,0.030769,0.015385,0.015385,0.000000,0.107692,0.015385,0.015385,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046154,0.015385,0.000000,0.015385,0.015385,0.000000,0.015385,0.030769,0.000000,0.046154,0.015385,0.000000,0.015385,0.107692,0.015385,0.015385,0.000000,0.000000,0.015385,0.030769,0.000000,0.030769,0.000000,0.046154,0.000000,0.030769,0.000000,0.030769,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.030769,0.030769,0.015385,0.046154,0.030769,0.015385,0.0000
1,Doll Asylum,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.066667,0.000000,0.066667,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.066667,0.0000
2,Harry C. Kendall Planetarium,0.100000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.100000,0.000000,0.000000,0.000000,0.000000,0.033333,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.033333,0.000000,0.0000
3,Hat Museum,0.046512,0.000000,0.000000,0.000000,0.000000,0.093023,0.046512,0.000000,0.023256,0.046512,0.000000,0.000000,0.000000,0.000000,0.069767,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046512,0.000000,0.023256,0.046512,0.000000,0.000000,0.000000,0.023256,0.000000,0.093023,0.046512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.093023,0.023256,0.000000,0.023256,0.000000,0.023256,0.069767,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.069767,0.000000,0.000000,0.023256,0.023256,0.023256,0.0000
4,Historic Belmont Firehouse,0.039216,0.019608,0.039216,0.000000,0.000000,0.078431,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.039216,0.019608,0.000000,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.019608,0.058824,0.000000,0.000000,0.000000,0.000000,0.039216,0.019608,0.000000,0.000000,0.039216,0.039216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.019608,0.000000,0.058824,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.039216,0.000000,0.000000,0.137255,0.058824,0.058824,0.0000
5,History Hub,0.0886

In [99]:
num_top_venues = 5

for FUN in Fun_Food_grouped['Fun_Venue']:
    print("----"+FUN+"----")
    temp = Fun_Food_grouped[Fun_Food_grouped['Fun_Venue'] == FUN].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Architectural Heritage Center----
                 venue  freq
0   Mexican Restaurant  0.11
1                 Café  0.11
2  Japanese Restaurant  0.05
3    French Restaurant  0.05
4      Thai Restaurant  0.05


----Doll Asylum----
                  venue  freq
0           Pizza Place  0.13
1  Ethiopian Restaurant  0.07
2              Creperie  0.07
3                  Food  0.07
4     Fish & Chips Shop  0.07


----Harry C. Kendall Planetarium----
                 venue  freq
0   Seafood Restaurant  0.13
1  American Restaurant  0.10
2   Italian Restaurant  0.10
3                 Café  0.10
4   Mexican Restaurant  0.10


----Hat Museum----
                venue  freq
0      Breakfast Spot  0.09
1  Mexican Restaurant  0.09
2         Pizza Place  0.09
3      Sandwich Place  0.07
4    Sushi Restaurant  0.07


----Historic Belmont Firehouse----
               venue  freq
0    Thai Restaurant  0.14
1     Breakfast Spot  0.08
2         Restaurant  0.06
3        Pizza Place  0.06
4  Indian Re

In [100]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Fun_Venue']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Fun_Food_venues_sorted = pd.DataFrame(columns=columns)
Fun_Food_venues_sorted['Fun_Venue'] = Fun_Food_grouped['Fun_Venue']

for ind in np.arange(Fun_Food_grouped.shape[0]):
    Fun_Food_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Fun_Food_grouped.iloc[ind, :], num_top_venues)

Fun_Food_venues_sorted.head()


,Fun_Venue,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Architectural Heritage Center,Café,Mexican Restaurant,Japanese Restaurant,Thai Restaurant,French Restaurant,Ramen Restaurant,Sandwich Place,Breakfast Spot,Deli / Bodega,Italian Restaurant
1,Doll Asylum,Pizza Place,Diner,Chinese Restaurant,Indian Restaurant,Thai Restaurant,BBQ Joint,Taco Place,Vietnamese Restaurant,Ethiopian Restaurant,Mediterranean Restaurant
2,Harry C. Kendall Planetarium,Seafood Restaurant,American Restaurant,Mexican Restaurant,Café,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Restaurant,Pizza Place,Sandwich Place
3,Hat Museum,Pizza Place,Mexican Restaurant,Breakfast Spot,Sandwich Place,Sushi Restaurant,Deli / Bodega,Indian Restaurant,Italian Restaurant,Middle Eastern Restaurant,Café
4,Historic Belmont Firehouse,Thai Restaurant,Breakfast Spot,Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,Vietnamese Restaurant,Pizza Place,Japanese Restaurant,BBQ Joint,Diner


In [102]:
# set number of clusters
kclusters = 5

Fun_Food_grouped_clustering = Fun_Food_grouped.drop('Fun_Venue', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Fun_Food_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 4, 0, 2, 4, 3, 2, 0, 3, 3])

In [103]:
#The Pittock Mansion has to be removed, or it makes havoc with the Clustering
fun_venues_wo_PM = fun_venues.drop(1)

In [104]:
# add clustering labels
Fun_Food_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Fun_Food_merged = fun_venues_wo_PM

# merge fun_venues with Fun_Food_venues_sorted to add latitude/longitude for each Fun Venue
Fun_Food_merged = Fun_Food_merged.join(Fun_Food_venues_sorted.set_index('Fun_Venue'), on='name')

Fun_Food_merged.head()


,name,categories,lat,lng,address,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Portland Art Museum,Art Museum,45.516230,-122.683597,1219 SW Park Ave,43ee1bd9f964a5205f2f1fe3,3,American Restaurant,Thai Restaurant,Seafood Restaurant,Café,Italian Restaurant,Pizza Place,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,French Restaurant
2,Oregon Museum of Science & Industry (OMSI),Science Museum,45.508601,-122.665915,1945 SE Water Ave,415c9e00f964a520551d1fe3,0,Seafood Restaurant,Mexican Restaurant,Café,American Restaurant,Sushi Restaurant,Italian Restaurant,Deli / Bodega,Restaurant,Pizza Place,Sandwich Place
3,Portland Children's Museum,Science Museum,45.508530,-122.717786,4015 SW Canyon Rd,4efdf6cea69d45461bd1cf1a,1,Café,Restaurant,Wings Joint,Israeli Restaurant,Irish Pub,Indian Restaurant,Hot Dog Joint,Greek Restaurant,Gastropub,French Restaurant
4,Oregon Historical Society,History Museum,45.515914,-122.682456,1200 SW Park Ave,4b5ce375f964a5207a4929e3,3,American Restaurant,Seafood Restaurant,Café,Italian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Thai Restaurant,Pizza Place,French Restaurant,Mexican Restaurant
5,Oregon History Museum,History Museum,45.515856,-122.682378,1211 SW Broadway,4ace6702f964a5205fd020e3,3,American Restaurant,Seafood Restaurant,Café,Italian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Thai Restaurant,Pizza Place,French Restaurant,Mexican Restaurant


In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Fun_Food_merged['lat'], Fun_Food_merged['lng'], Fun_Food_merged['name'], Fun_Food_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-5],
        fill=True,
        fill_color=rainbow[cluster-5],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [106]:
Fun_Food_merged.loc[Fun_Food_merged['Cluster Labels'] == 0, Fun_Food_merged.columns[[1] + list(range(5, Fun_Food_merged.shape[1]))]]

,categories,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Science Museum,415c9e00f964a520551d1fe3,0,Seafood Restaurant,Mexican Restaurant,Café,American Restaurant,Sushi Restaurant,Italian Restaurant,Deli / Bodega,Restaurant,Pizza Place,Sandwich Place
6,Planetarium,4d72cffb8e12b1f7d7764105,0,Seafood Restaurant,American Restaurant,Mexican Restaurant,Café,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Restaurant,Pizza Place,Sandwich Place
7,Boat or Ferry,4b48e783f964a520a85c26e3,0,Seafood Restaurant,Mexican Restaurant,Café,Sushi Restaurant,Deli / Bodega,American Restaurant,Italian Restaurant,Sandwich Place,Pizza Place,Vegetarian / Vegan Restaurant
11,General Entertainment,4d940677578ba093caf69a57,0,Seafood Restaurant,Mexican Restaurant,Café,Deli / Bodega,Italian Restaurant,Sushi Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Mediterranean Restaurant
12,Science Museum,4c250ed2f7ced13afce6236d,0,Seafood Restaurant,Mexican Restaurant,Café,Sushi Restaurant,Deli / Bodega,American Restaurant,Italian Restaurant,Sandwich Place,Pizza Place,Vegetarian / Vegan Restaurant
14,Museum,5ab3e3afa4ba7c0787f37f21,0,Seafood Restaurant,Mexican Restaurant,Café,Sushi Restaurant,Deli / Bodega,American Restaurant,Italian Restaurant,Sandwich Place,Pizza Place,Vegetarian / Vegan Restaurant
18,Science Museum,4e33631a483b5fa5874c5721,0,Seafood Restaurant,Mexican Restaurant,Café,Deli / Bodega,Italian Restaurant,Sushi Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Mediterranean Restaurant
25,Museum,5c0d568e6a5950002ce7bb67,0,Seafood Restaurant,Mexican Restaurant,Café,Sushi Restaurant,Deli / Bodega,American Restaurant,Italian Restaurant,Sandwich Place,Pizza Place,Vegetarian / Vegan Restaurant


In [107]:
Fun_Food_merged.loc[Fun_Food_merged['Cluster Labels'] == 1, Fun_Food_merged.columns[[1] + list(range(5, Fun_Food_merged.shape[1]))]]

,categories,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Science Museum,4efdf6cea69d45461bd1cf1a,1,Café,Restaurant,Wings Joint,Israeli Restaurant,Irish Pub,Indian Restaurant,Hot Dog Joint,Greek Restaurant,Gastropub,French Restaurant
10,Museum,49cc4846f964a5205e591fe3,1,Café,Restaurant,Wings Joint,Israeli Restaurant,Irish Pub,Indian Restaurant,Hot Dog Joint,Greek Restaurant,Gastropub,French Restaurant


In [108]:
Fun_Food_merged.loc[Fun_Food_merged['Cluster Labels'] == 2, Fun_Food_merged.columns[[1] + list(range(5, Fun_Food_merged.shape[1]))]]

,categories,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Train Station,505e2e16e4b0ddb6e8ee4713,2,Breakfast Spot,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Restaurant,Sandwich Place,Italian Restaurant,Vietnamese Restaurant,Pizza Place
9,Art Museum,4b25c71df964a5201a7624e3,2,Mexican Restaurant,Sandwich Place,Breakfast Spot,American Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Café,Deli / Bodega,Italian Restaurant,Vietnamese Restaurant
17,History Museum,4b25d8c1f964a520dd7624e3,2,Pizza Place,Mexican Restaurant,Breakfast Spot,Sandwich Place,Sushi Restaurant,Deli / Bodega,Indian Restaurant,Italian Restaurant,Middle Eastern Restaurant,Café
21,History Museum,4ba3b847f964a520145838e3,2,Café,Mexican Restaurant,Japanese Restaurant,Thai Restaurant,French Restaurant,Ramen Restaurant,Sandwich Place,Breakfast Spot,Deli / Bodega,Italian Restaurant
22,Museum,5708625a498e6e5b9304f48f,2,Pizza Place,Thai Restaurant,Mexican Restaurant,Café,Sandwich Place,Deli / Bodega,Japanese Restaurant,Seafood Restaurant,Ramen Restaurant,Italian Restaurant


In [109]:
Fun_Food_merged.loc[Fun_Food_merged['Cluster Labels'] == 3, Fun_Food_merged.columns[[1] + list(range(5, Fun_Food_merged.shape[1]))]]

,categories,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Art Museum,43ee1bd9f964a5205f2f1fe3,3,American Restaurant,Thai Restaurant,Seafood Restaurant,Café,Italian Restaurant,Pizza Place,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,French Restaurant
4,History Museum,4b5ce375f964a5207a4929e3,3,American Restaurant,Seafood Restaurant,Café,Italian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Thai Restaurant,Pizza Place,French Restaurant,Mexican Restaurant
5,History Museum,4ace6702f964a5205fd020e3,3,American Restaurant,Seafood Restaurant,Café,Italian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Thai Restaurant,Pizza Place,French Restaurant,Mexican Restaurant
13,History Museum,4c4f6c1163690f47f6f66946,3,Sandwich Place,Mexican Restaurant,American Restaurant,Restaurant,Asian Restaurant,Burger Joint,Japanese Restaurant,Middle Eastern Restaurant,Thai Restaurant,Steakhouse
15,Library,5ad8dda4840fc27a8b6113c1,3,American Restaurant,Thai Restaurant,Café,Italian Restaurant,Sandwich Place,Seafood Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,French Restaurant,Mexican Restaurant
16,History Museum,4b2daf49f964a52062da24e3,3,American Restaurant,Burger Joint,Restaurant,Asian Restaurant,Café,Pizza Place,Sandwich Place,Mexican Restaurant,Steakhouse,Diner
19,History Museum,5bb527a91c675b002c874c24,3,American Restaurant,Pizza Place,Burger Joint,Asian Restaurant,Sandwich Place,Mexican Restaurant,Vegetarian / Vegan Restaurant,Diner,Restaurant,Salad Place
24,History Museum,5ad8e545b9ac383a1de79567,3,American Restaurant,Thai Restaurant,Café,Italian Restaurant,Sandwich Place,Seafood Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,French Restaurant,Mexican Restaurant
27,History Museum,4be3002e660ec92878d9ca3b,3,American Restaurant,Pizza Place,Burger Joint,Italian Restaurant,Sandwich Place,Diner,Mexican Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant


In [110]:
Fun_Food_merged.loc[Fun_Food_merged['Cluster Labels'] == 4, Fun_Food_merged.columns[[1] + list(range(5, Fun_Food_merged.shape[1]))]]

,categories,id,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Museum,544da6e3498e57364d2e0541,4,Pizza Place,Diner,Chinese Restaurant,Indian Restaurant,Thai Restaurant,BBQ Joint,Taco Place,Vietnamese Restaurant,Ethiopian Restaurant,Mediterranean Restaurant
23,History Museum,53f53d06498e31b96a2ad5b6,4,Thai Restaurant,Mexican Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Diner,Sandwich Place,Indian Restaurant,Breakfast Spot,Restaurant,Italian Restaurant
26,History Museum,5287f52411d235ac6c3f789f,4,Thai Restaurant,Breakfast Spot,Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,Vietnamese Restaurant,Pizza Place,Japanese Restaurant,BBQ Joint,Diner
